## Filters

Filters return a boolean value. We can use them as the name suggest to 'filter' our output. For example getting all the stock that trade above \\$50.

Next we will build some filters in our pipeline output.

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.data import USEquityPricing

### Comparison operators

This operation will help us compare two factors and return a value that meets certain criteria.

In [2]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    
    mean_close_50 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 50
    )
    
    mean_close_200 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 200
    )
    
    mean_crossover_filter = mean_close_50 > mean_close_200
    
    return Pipeline(
    
        columns = {
            
            'mean_close_50':mean_close_50,
            'mean_close_200':mean_close_200,
            'latest_close':latest_close,
            'crossover':mean_crossover_filter
        }
    )
    
    
    

In [3]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head(10)

crossover  latest_close  \
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])         False         8.068   
                          Equity(21 [AAME])         True         4.460   
                          Equity(24 [AAPL])        False        96.980   
                          Equity(25 [ARNC_PR])      True           NaN   
                          Equity(31 [ABAX])         True        51.480   
                          Equity(39 [DDC])         False        10.090   
                          Equity(41 [ARCB])        False        18.660   
                          Equity(52 [ABM])         False        26.600   
                          Equity(53 [ABMD])         True        84.640   
                          Equity(62 [ABT])         False        40.650   

                                                mean_close_200  mean_close_50  
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])           10.580066       9.056216  
                          Equity(21 [AAME])           4.034396       4.758100  
                          Equity(24 [AAPL])         118.676494     113.593691  
                          Equity(25 [ARNC_PR])       83.433936      84.467737  
                          Equity(31 [ABAX])          52.082381      53.170782  
                          Equity(39 [DDC])           13.027227       9.208280  
                          Equity(41 [ARCB])          29.940789      23.286936  
                          Equity(52 [ABM])           30.510478      28.615897  
                          Equity(53 [ABMD])          79.811790      83.191900  
                          Equity(62 [ABT])           46.039930      44.827800

### Factor/Classifier Method

Sometimes we would like to pick the top or bottom 100 stocks. We can use the factor/classifier method to perform this action. The below code will return `True` when the stock's closing price is in the top 100 and return `False` otherwise.

In [4]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    
    top_close_price = latest_close.top(100)
    
    return Pipeline(
    
        columns = {
            
            'top_close_price':top_close_price,
            'latest_close':latest_close
        }
    )


    
    

In [5]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head(10)

latest_close top_close_price
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])             8.068           False
                          Equity(21 [AAME])            4.460           False
                          Equity(24 [AAPL])           96.980           False
                          Equity(25 [ARNC_PR])           NaN           False
                          Equity(31 [ABAX])           51.480           False
                          Equity(39 [DDC])            10.090           False
                          Equity(41 [ARCB])           18.660           False
                          Equity(52 [ABM])            26.600           False
                          Equity(53 [ABMD])           84.640           False
                          Equity(62 [ABT])            40.650           False

Lets look at some values that returned `True`

In [6]:
output[output['top_close_price']].head(10)

latest_close top_close_price
2016-01-11 00:00:00+00:00 Equity(332 [ALX])           374.77            True
                          Equity(368 [AMGN])          150.70            True
                          Equity(693 [AZO])           712.27            True
                          Equity(779 [BCR])           180.09            True
                          Equity(1091 [BRK_A])     193680.00            True
                          Equity(1216 [CACC])         206.40            True
                          Equity(1753 [COKE])         177.51            True
                          Equity(1787 [COST])         152.11            True
                          Equity(2174 [DIA])          163.26            True
                          Equity(2194 [DJCO])         192.70            True

### Dollar Volume filter

We can also create filters to select only the stocks that have had 30 day average dollar volume above \\$10million

In [7]:
from quantopian.pipeline.factors import AverageDollarVolume

def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    
    mean_close_50 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 50
    )
    
    mean_close_200 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 200
    )
    
    mean_crossover_filter = mean_close_50 > mean_close_200
    
    dollar_volume = AverageDollarVolume(window_length = 30)
    high_dollar_volume = dollar_volume > 10e6
    
    return Pipeline(
    
        columns = {
            
            'mean_close_50':mean_close_50,
            'mean_close_200':mean_close_200,
            'latest_close':latest_close,
            'crossover':mean_crossover_filter,
            'high_dollar_volume':high_dollar_volume
        }
    )
    
    

In [8]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head(10)

crossover high_dollar_volume  \
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])         False               True   
                          Equity(21 [AAME])         True              False   
                          Equity(24 [AAPL])        False               True   
                          Equity(25 [ARNC_PR])      True              False   
                          Equity(31 [ABAX])         True              False   
                          Equity(39 [DDC])         False              False   
                          Equity(41 [ARCB])        False              False   
                          Equity(52 [ABM])         False              False   
                          Equity(53 [ABMD])         True               True   
                          Equity(62 [ABT])         False               True   

                                                latest_close  mean_close_200  \
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])             8.068       10.580066   
                          Equity(21 [AAME])            4.460        4.034396   
                          Equity(24 [AAPL])           96.980      118.676494   
                          Equity(25 [ARNC_PR])           NaN       83.433936   
                          Equity(31 [ABAX])           51.480       52.082381   
                          Equity(39 [DDC])            10.090       13.027227   
                          Equity(41 [ARCB])           18.660       29.940789   
                          Equity(52 [ABM])            26.600       30.510478   
                          Equity(53 [ABMD])           84.640       79.811790   
                          Equity(62 [ABT])            40.650       46.039930   

                                                mean_close_50  
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])           9.056216  
                          Equity(21 [AAME])          4.758100  
                          Equity(24 [AAPL])        113.593691  
                          Equity(25 [ARNC_PR])      84.467737  
                          Equity(31 [ABAX])         53.170782  
                          Equity(39 [DDC])           9.208280  
                          Equity(41 [ARCB])         23.286936  
                          Equity(52 [ABM])          28.615897  
                          Equity(53 [ABMD])         83.191900  
                          Equity(62 [ABT])          44.827800

### Applying the Screen

So whats the point of getting the `True` or `False` values? It is used to apply a screen in our pipeline. Suppose we are only interested in the high_dollar_volume stocks. Then we can use that as a screen.

In [9]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    
    mean_close_50 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 50
    )
    
    mean_close_200 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 200
    )
    
    mean_crossover_filter = mean_close_50 > mean_close_200
    
    dollar_volume = AverageDollarVolume(window_length = 30)
    high_dollar_volume = dollar_volume > 10e6
    
    return Pipeline(
    
        columns = {
            
            'mean_close_50':mean_close_50,
            'mean_close_200':mean_close_200,
            'latest_close':latest_close,
            'crossover':mean_crossover_filter,
            'dollar_volume':dollar_volume
        }, screen = high_dollar_volume
    )

In [10]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head(10)

crossover  dollar_volume  \
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])       False   2.274140e+08   
                          Equity(24 [AAPL])      False   3.984098e+09   
                          Equity(53 [ABMD])       True   2.731392e+07   
                          Equity(62 [ABT])       False   2.069585e+08   
                          Equity(64 [GOLD])      False   1.152668e+08   
                          Equity(67 [ADSK])       True   9.247820e+07   
                          Equity(76 [TAP])        True   1.054230e+08   
                          Equity(114 [ADBE])      True   1.803706e+08   
                          Equity(122 [ADI])      False   1.193104e+08   
                          Equity(128 [ADM])      False   1.387628e+08   

                                              latest_close  mean_close_200  \
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])           8.068       10.580066   
                          Equity(24 [AAPL])         96.980      118.676494   
                          Equity(53 [ABMD])         84.640       79.811790   
                          Equity(62 [ABT])          40.650       46.039930   
                          Equity(64 [GOLD])          8.410        9.008780   
                          Equity(67 [ADSK])         55.070       55.289560   
                          Equity(76 [TAP])          89.190       78.621830   
                          Equity(114 [ADBE])        87.840       82.998900   
                          Equity(122 [ADI])         49.980       59.710895   
                          Equity(128 [ADM])         34.390       44.669667   

                                              mean_close_50  
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])         9.056216  
                          Equity(24 [AAPL])      113.593691  
                          Equity(53 [ABMD])       83.191900  
                          Equity(62 [ABT])        44.827800  
                          Equity(64 [GOLD])        7.521845  
                          Equity(67 [ADSK])       61.210240  
                          Equity(76 [TAP])        91.888813  
                          Equity(114 [ADBE])      91.503300  
                          Equity(122 [ADI])       57.864331  
                          Equity(128 [ADM])       37.607770

In [11]:
print("The number of securities that passed the High Dollar Volume filter : %d" % len(output))

The number of securities that passed the High Dollar Volume filter : 1993


### Inverting a Filter

Lets do the opposite. We will filter and select the low dollar volume stocks using the `~` operator.

In [12]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    
    mean_close_50 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 50
    )
    
    mean_close_200 = SimpleMovingAverage(
    
        inputs = [USEquityPricing.close],
        window_length = 200
    )
    
    mean_crossover_filter = mean_close_50 > mean_close_200
    
    dollar_volume = AverageDollarVolume(window_length = 30)
    high_dollar_volume = dollar_volume > 10e6
    
    return Pipeline(
    
        columns = {
            
            'mean_close_50':mean_close_50,
            'mean_close_200':mean_close_200,
            'latest_close':latest_close,
            'crossover':mean_crossover_filter,
            'dollar_volume':dollar_volume
        }, screen = ~high_dollar_volume
    )

In [13]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head(10)

crossover  dollar_volume  \
2016-01-11 00:00:00+00:00 Equity(21 [AAME])         True   1.825666e+04   
                          Equity(25 [ARNC_PR])      True   2.021752e+04   
                          Equity(31 [ABAX])         True   5.272874e+06   
                          Equity(39 [DDC])         False   4.134326e+06   
                          Equity(41 [ARCB])        False   5.136647e+06   
                          Equity(52 [ABM])         False   5.850535e+06   
                          Equity(66 [AB])          False   7.410636e+06   
                          Equity(69 [ACAT])        False   3.105641e+06   
                          Equity(70 [VBF])          True   3.930920e+05   
                          Equity(84 [ACET])         True   6.210320e+06   

                                                latest_close  mean_close_200  \
2016-01-11 00:00:00+00:00 Equity(21 [AAME])             4.46        4.034396   
                          Equity(25 [ARNC_PR])           NaN       83.433936   
                          Equity(31 [ABAX])            51.48       52.082381   
                          Equity(39 [DDC])             10.09       13.027227   
                          Equity(41 [ARCB])            18.66       29.940789   
                          Equity(52 [ABM])             26.60       30.510478   
                          Equity(66 [AB])              21.84       27.155139   
                          Equity(69 [ACAT])            14.34       27.365528   
                          Equity(70 [VBF])             17.63       17.663242   
                          Equity(84 [ACET])            22.15       24.984356   

                                                mean_close_50  
2016-01-11 00:00:00+00:00 Equity(21 [AAME])          4.758100  
                          Equity(25 [ARNC_PR])      84.467737  
                          Equity(31 [ABAX])         53.170782  
                          Equity(39 [DDC])           9.208280  
                          Equity(41 [ARCB])         23.286936  
                          Equity(52 [ABM])          28.615897  
                          Equity(66 [AB])           24.392800  
                          Equity(69 [ACAT])         19.122425  
                          Equity(70 [VBF])          17.950518  
                          Equity(84 [ACET])         27.147669

In [14]:
print("The number of securities that passed the Low Dollar Volume filter : %d" % len(output))

The number of securities that passed the Low Dollar Volume filter : 6408
